In [11]:
import sqlite3 as lite
import pandas as pd

db = lite.connect('schemaperfect.db')

In [21]:
query = """
    select name
    from sqlite_master
    where
        type='table'
    ;
"""
pd.read_sql(query,db)

,name
0,Categories
1,Customers
2,Employees
3,Shippers
4,Suppliers
5,Orders
6,OrderDetails
7,Products


In [24]:
pd.options.display.max_colwidth=100
query = """
    select sql
    from sqlite_master
    where
        type='table'
    ;
"""
pd.read_sql(query,db)

,sql
0,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ..."
2,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""Birth..."
3,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
4,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ..."
5,"CREATE TABLE Orders(\nOrderID integer,\nCustomerID integer,\nEmployeeID integer,\nOrderDate date..."
6,"CREATE TABLE OrderDetails(\nOrderDetailID integer,\nOrderID integer,\nProductID integer,\nQuanti..."
7,"CREATE TABLE Products(\nProductID integer,\nProductName text,\nSupplierID integer,\nCategoryID i..."


In [38]:
query ="""
    select *
    from Customers
    where 
        Country='France'
        or Country="Mexico"
    order by
        Address desc
    ;
"""
pd.read_sql(query,db).head(2)

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,13,Centro comercial Moctezuma,Francisco Chang,Sierras de Granada 9993,México D.F.,05022,Mexico
1,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico


In [51]:
query ="""
    select *
    from Orders O
    ;
"""
pd.read_sql(query,db).head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [53]:
# Group by
query ="""
    select *
    from Orders
    group by OrderDate
    ;
"""
pd.read_sql(query,db).head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10251,84,3,1996-07-08,1
3,10252,76,4,1996-07-09,2
4,10253,34,3,1996-07-10,2


In [74]:
query ="""
    select *
    from Orders
    Where
        OrderDate like "%-01-%"
        and ShipperID in (1, 3)
    group by 
        OrderDate
    ;
"""
pd.read_sql(query,db).head(10)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10401,65,1,1997-01-01,1
1,10404,49,2,1997-01-03,1
2,10405,47,1,1997-01-06,1
3,10406,62,7,1997-01-07,1
4,10408,23,8,1997-01-08,1
5,10409,54,3,1997-01-09,1
6,10411,10,9,1997-01-10,3
7,10414,21,2,1997-01-14,3
8,10415,36,3,1997-01-15,1
9,10417,73,4,1997-01-16,3


In [87]:
query ="""
    select
         Count(*) "Count ", O.OrderDate
    from Orders O
    Join
        Customers C
        On O.CustomerID = C.CustomerID
    Group BY
        OrderDate
    ;
"""
pd.read_sql(query,db).head(10)

,Count,OrderDate
0,1,1996-07-04
1,1,1996-07-05
2,2,1996-07-08
3,1,1996-07-09
4,1,1996-07-10
5,1,1996-07-11
6,1,1996-07-12
7,1,1996-07-15
8,1,1996-07-16
9,1,1996-07-17


In [100]:
query ="""
    select 
         Count(*) "Count ", substr(O.OrderDate,0,8) "OrderMonth"
    from Orders O
    Join
        Customers C
        On O.CustomerID = C.CustomerID
    Group BY
        OrderMonth
    ;
"""
pd.read_sql(query,db).head()

,Count,OrderMonth
0,22,1996-07
1,25,1996-08
2,23,1996-09
3,26,1996-10
4,25,1996-11


In [107]:
query ="""
    select round(OD.Quantity * P.Price,1) "Total"
    from OrderDetails OD
    Join
        Products P
        On OD.ProductID = P.ProductID
    ;
"""
pd.read_sql(query,db).head()

,Total
0,252.0
1,140.0
2,174.0
3,209.2
4,2120.0


In [116]:
# 월별 총 매출과 건당 평균매출

query ="""
    select OD.OrderID, OD.ProductID, OD.Quantity, substr(O.OrderDate,0,8) "Month", P.Price
    from OrderDetails OD
    Join
        Orders O
        ON O.OrderID = OD.OrderID
    Join
        Products P
        On P.ProductID=OD.ProductID
    Group by
        Month
    ;
"""
pd.read_sql(query,db).head()

,OrderID,ProductID,Quantity,Month,Price
0,10269,72,20,1996-07,34.80
1,10294,75,6,1996-08,7.75
2,10317,1,20,1996-09,18.00
3,10343,76,15,1996-10,18.00
4,10368,64,35,1996-11,33.25


In [119]:
# total sales a month and avg sales per sale

query ="""
    select 
        substr(O.OrderDate,0,8) "Month",
        sum(OD.Quantity*P.Price) "TotalSales",
        avg(OD.Quantity*P.Price) "AvgSales",
        Count(*) "Count"
    from OrderDetails OD
    Join
        Orders O
        ON O.OrderID = OD.OrderID
    Join
        Products P
        On P.ProductID=OD.ProductID
    Group by
        Month
    ;
"""
pd.read_sql(query,db).head()

,Month,TotalSales,AvgSales,Count
0,1996-07,37779.85,640.336441,59
1,1996-08,33285.49,482.398406,69
2,1996-09,34565.60,606.414035,57
3,1996-10,51528.69,705.872466,73
4,1996-11,62163.99,941.878636,66


In [127]:
# Where vs Having

query ="""
    select 
        substr(O.OrderDate,0,8) "Month",
        sum(OD.Quantity*P.Price) "TotalSales",
        avg(OD.Quantity*P.Price) "AvgSales",
        Count(*) "Count"
    from OrderDetails OD
    Join
        Orders O
        ON O.OrderID = OD.OrderID
    Join
        Products P
        On P.ProductID=OD.ProductID
    where
        OD.Quantity>=20
    Group by
        Month
    ;
"""
pd.read_sql(query,db).head()

,Month,TotalSales,AvgSales,Count
0,1996-07,31058.15,913.475000,34
1,1996-08,21749.75,701.604839,31
2,1996-09,28332.05,976.967241,29
3,1996-10,44719.02,1146.641538,39
4,1996-11,53621.95,1449.241892,37


In [128]:
# Having

query ="""
    select 
        substr(O.OrderDate,0,8) "Month",
        sum(OD.Quantity*P.Price) "TotalSales",
        avg(OD.Quantity*P.Price) "AvgSales",
        Count(*) "Count"
    from OrderDetails OD
    Join
        Orders O
        ON O.OrderID = OD.OrderID
    Join
        Products P
        On P.ProductID=OD.ProductID
    Group by
        Month
    Having
        OD.Quantity>=20
    ;
"""
pd.read_sql(query,db).head()

,Month,TotalSales,AvgSales,Count
0,1996-07,37779.85,640.336441,59
1,1996-09,34565.60,606.414035,57
2,1996-11,62163.99,941.878636,66
3,1997-01,83400.47,981.182000,85
